In [48]:
import pandas as pd
import requests
import json
import numpy as np

# Setting  up

In [3]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('BMqT9-TtQsS95OJUScGvGw', 'Y3FARMDLa1zm2NcbwAwuNrJR_IdNag')

In [4]:
# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'botstudy/0.0.1'}

In [5]:
# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'softyarn',
        'password': '1532679!Qa1',
        'User-Agent':'botstudy/0.0.1'}

In [6]:
# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

In [7]:
# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

In [8]:
# convert response to JSON and pull access_token value
res.json()

{'access_token': '167791397396-cl1mspkC7aP6Zi6M2eQBTddLJq4VtQ',
 'token_type': 'bearer',
 'expires_in': 86400,
 'scope': '*'}

In [9]:
# convert response to JSON and pull access_token value
token = res.json()['access_token']

In [10]:
# add authorization to our headers dictionary
headers['Authorization'] = 'bearer {}'.format(token)

In [11]:
# while the token is valid (~2 hours) we just add headers=headers to our requests. End point: GET /api/v1/me
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).json()

{'is_employee': False,
 'seen_layout_switch': False,
 'has_visited_new_profile': False,
 'pref_no_profanity': True,
 'has_external_account': False,
 'pref_geopopular': '',
 'seen_redesign_modal': False,
 'pref_show_trending': True,
 'subreddit': {'default_set': False,
  'user_is_contributor': False,
  'banner_img': '',
  'restrict_posting': True,
  'user_is_banned': False,
  'free_form_reports': True,
  'community_icon': None,
  'show_media': True,
  'icon_color': '',
  'user_is_muted': None,
  'display_name': 'u_softyarn',
  'header_img': None,
  'title': '',
  'coins': 0,
  'previous_names': [],
  'over_18': False,
  'icon_size': [256, 256],
  'primary_color': '',
  'icon_img': 'https://styles.redditmedia.com/t5_3ef3ey/styles/profileIcon_snoo86ebddf0-4659-47c8-9ad6-b6ca0588943d-headshot-f.png?width=256&amp;height=256&amp;crop=256:256,smart&amp;s=86680ec95737b7ce10db712e2889672a4de8bbf1',
  'description': '',
  'allowed_media_in_comments': [],
  'submit_link_label': '',
  'header_size

# Miscellanous scraping

In [ ]:
#Get moderators of feminisms
fems= requests.get('https://oauth.reddit.com/r/feminisms/about/moderators', headers=headers).json()

In [ ]:
pd.DataFrame(fems['data']['children'])

# Get comments by name

## Write function

In [130]:
def comments(source_dir) :
    df = pd.read_csv(source_dir)
    df = df[["link_id","body"]]
    df["note"] = np.where(df["body"] == "[removed]", "removed", np.where(df["body"] == "[deleted]", "deleted", "text"))

    for i in range(len(df["link_id"])):
        x = ",".join(df["link_id"])
    #x = df["link_id"].to_string(header= False, index = False).split(",")
    #vals =  [','.join(ele.split()) for ele in x]
    return x

In [131]:
fds_list = data_prep(source_dir = "E:/gihub-data/redditbots/fds/fds_comments.csv")

C:\Users\nguye\AppData\Local\Temp\ipykernel_36428\3507331379.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(source_dir)


In [133]:
fds = pd.read_csv("E:/gihub-data/redditbots/fds/fds_comments.csv")
fds = fds[["link_id","body"]]
fds["note"] = np.where(fds["body"] == "[removed]", "removed", np.where(fds["body"] == "[deleted]", "deleted", "text"))

C:\Users\nguye\AppData\Local\Temp\ipykernel_36428\2950681155.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  fds = pd.read_csv("E:/gihub-data/redditbots/fds/fds_comments.csv")


In [148]:
for i in range(len(fds["link_id"])):
    x = ",".join(fds["link_id"][i:i+20])
    comments = requests.get('https://oauth.reddit.com/by_id/names',headers=headers,params={'names': x})
    i = i + 

In [147]:
comment_df = pd.DataFrame.from_dict(comments.json().get("data").get("children")[25].get("data"), orient = "index").transpose()

IndexError: list index out of range

In [140]:
comment_df

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,...,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video
0,None,FemaleDatingStrategy,"The older we get, the more scared of us they a...",t2_3t1jriw3,False,None,0,False,"Don't buy the ""women hit the wall at 25"" incel...",[],...,None,/r/FemaleDatingStrategy/comments/ceyqp6/dont_b...,no_ads,False,https://www.reddit.com/r/FemaleDatingStrategy/...,248208,1563486298.0,1,None,False


In [82]:
i = 0
for i in range(len(fds_list)):
    comments = requests.get('https://oauth.reddit.com/by_id/names',headers=headers,params={'names': fds_list[i:i+100]})
    
    df = pd.DataFrame()

    for x in range(i,i+100):
        comment = pd.DataFrame.from_dict(submissions.json().get("data").get("children")[i].get("data"), orient = "index").transpose()
        df = pd.concat([df,comment])


IndexError: list index out of range

In [127]:
comment_df = pd.DataFrame.from_dict(comments.json().get("data").get("children")[0].get("data"), orient = "index").transpose()

AttributeError: 'NoneType' object has no attribute 'get'

In [108]:
df = pd.DataFrame()
i = 0
for i in range(0,99):
    comment = pd.DataFrame.from_dict(comments.json().get("data").get("children")[i].get("data"), orient = "index").transpose()
    df = pd.concat([df,comment])

IndexError: list index out of range

In [119]:
comments.json()

{'kind': 'Listing',
 'data': {'after': None,
  'dist': 1,
  'modhash': None,
  'geo_filter': '',
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'FemaleDatingStrategy',
     'selftext': "The older we get, the more scared of us they are because we've learned not to fall for their bullshit. That's all it is.",
     'author_fullname': 't2_3t1jriw3',
     'saved': False,
     'mod_reason_title': None,
     'gilded': 0,
     'clicked': False,
     'title': 'Don\'t buy the "women hit the wall at 25" incel lie. It is a strategy to make women insecure and settle for losers. Older women are confident. Confidence is power.',
     'link_flair_richtext': [],
     'subreddit_name_prefixed': 'r/FemaleDatingStrategy',
     'hidden': False,
     'pwls': 0,
     'link_flair_css_class': None,
     'downs': 0,
     'thumbnail_height': None,
     'top_awarded_type': None,
     'hide_score': False,
     'name': 't3_ceyqp6',
     'quarantine': False,
     'link_flair_te

In [89]:
for i in range(0,100):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [20]:
submissions =  requests.get('https://oauth.reddit.com/by_id/names',headers=headers,params={'names':"t3_e4t8fw,t3_djaq2o,t3_d4b9y1", 'raw_json': '1'})

In [15]:
submissions

<Response [200]>

In [21]:
submissions_test = pd.DataFrame.from_dict(submissions.json().get("data").get("children")[1].get("data"), orient = "index").transpose()

In [22]:
submissions_test_2 = pd.DataFrame.from_dict(submissions.json().get("data").get("children")[2].get("data"), orient = "index").transpose()

In [23]:
pd.concat([submissions_test, submissions_test_2])

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,...,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video
0,None,FemaleDatingStrategy,"Have you ever seen an advice column, or flippe...",t2_3lobbi9k,False,None,0,False,"If you want commitment, marriage, children, to...",[],...,dark,/r/FemaleDatingStrategy/comments/djaq2o/if_you...,no_ads,False,https://www.reddit.com/r/FemaleDatingStrategy/...,248173,1571337946.0,0,None,False
0,None,FemaleDatingStrategy,\n\n# \n\n&#x200B;\n\nNever beg for a man’s t...,t2_3xtpll3y,False,None,0,False,"Replace, don't chase",[],...,None,/r/FemaleDatingStrategy/comments/d4b9y1/replac...,no_ads,False,https://www.reddit.com/r/FemaleDatingStrategy/...,248173,1568496311.0,0,None,False


In [ ]:
pd.concat([comment_test, comment_test_2]).to_csv("E:/gihub-data/redditbots/test.csv")

In [ ]:
type(comment_test)